In [1]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
from helpers import TrainingMonitor
from helpers import Utils

In [3]:
output_path = "../output/"

In [4]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [5]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

In [6]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

mean = np.mean(x_train, axis=0)
x_train -= mean
x_test -= mean

lb = LabelBinarizer()
y_train = lb.fit_transform(y_train) 
y_test = lb.fit_transform(y_test)

In [7]:
db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

In [16]:
json_file = open(output_path + 'saved/vgg_base_model_86.03.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)
model.load_weights(output_path + "saved/vgg_base_weight_86.03.hdf5")

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_2 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        18496     
__________

In [18]:
for (i, layer) in enumerate(model.layers): 
    print("{}\t{}".format(i, layer.__class__.__name__))

0	Conv2D
1	BatchNormalization
2	Conv2D
3	BatchNormalization
4	MaxPooling2D
5	Dropout
6	Conv2D
7	BatchNormalization
8	Conv2D
9	BatchNormalization
10	MaxPooling2D
11	Dropout
12	Conv2D
13	BatchNormalization
14	Conv2D
15	BatchNormalization
16	MaxPooling2D
17	Dropout
18	Flatten
19	Dense
20	BatchNormalization
21	Dense
22	BatchNormalization
23	Dense


In [31]:
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-vgg.png', show_shapes=True, show_layer_names=True)

In [20]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
filepath=output_path + "progress/weights-{val_acc:.4f}.hdf5"
MC = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')

figPath = os.path.sep.join([output_path, "monitor/{}.png".format(os.getpid())])
jsonPath = os.path.sep.join([output_path, "monitor/{}.json".format(os.getpid())])
TM = TrainingMonitor(figPath, jsonPath=jsonPath, startAt=0)

RLR = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)

callbacks = [MC, TM, RLR]

In [30]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=1,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

Train on 33500 samples, validate on 16500 samples
Epoch 1/1
33500/33500 [==============================] - 15s 438us/step - loss: 0.1268 - acc: 0.9597 - val_loss: 0.1164 - val_acc: 0.9643


In [28]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )

Train: 95.85%; Val: 96.32%; Test: 96.30%
